In [21]:
import sys
import warnings
import pandas as pd
import numpy as np
import os
import cx_Oracle as O
import datetime as dt
import logging, logging.handlers, smtplib
import traceback
import smtplib
import sqlalchemy as sa
import pyodbc
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from os.path import basename
import os
from datetime import datetime, date
import settings
import errhandler as eh
import eHSQL
import re
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

con = None
try:
    global conn, recordlist
    conn = False
    logging.basicConfig(filename = 'eHARSExtract.log',
                       filemode='a',
                       format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                       datefmt='%H:%M:%S',
                       level=logging.DEBUG)
    logging.info(f'\nLOG START: {datetime.now()}\n')

    #Opens connection to the Oracle DB via the stored config file in the config folder
    def open_connection():
        global driver, database, uid, pwd, connection, con
        #connection = 'DRIVER={Oracle in instantclient_21_6};DBQ=NDEDSS_DEVsvc.itd.nd.gov;UID=NDDH_EDSS_Intr;PWD=QRSTUVW1'
        connection = f'{settings.username}/{settings.password}@{settings.servicename}'
        con = O.connect(connection)
        #con = pyodbc.connect(connection)
        return con

    def connection_test():
        cur = con.cursor()
        cur.execute("SELECT * FROM v$version")
        row = cur.fetchone()
        print("Connection established to: ", row[0])
        cur.close()
        con.commit()
        logging.info(f"\nConnection established to: {row[0]}\n")
        return

    def close_connection():
        con.close()
        return

    def tablecount(con):
        cur = con.cursor()
        cur.execute(eHSQL.SQLCaseCount)
        output = cur.fetchone()
        #print(output)
        if output[0] == 0:
            print(f'\n{output[0]} Existing Records\n', end = "\r")
            logging.info(f'\n{output[0]} Existing records\n')
            cur.close()
            con.commit()
            return False
        else:
            print(f'\n{output[0]} Existing Records\n', end = "\r")
            logging.info(f'\n{output[0]} Existing records\n')
            cur.close()
            con.commit()
            return True

    def tablecontent(con):
        global recordlist
        cur = con.cursor()
        SQL = eHSQL.SQLHIVData
        cur.execute(SQL)
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            df = pd.read_sql(SQL, con)
            display(df)
        output = cur.fetchall()
        output = output[0:]
        output = str(tuple(list(list(zip(*output))[0])))
        #print(output)
        #print(f'\nRecord List: {output}\n', end = "\r")
        logging.info(f'\nRecord List: {output}\n')
        recordlist = output
        cur.close()
        con.commit()
        return

    def get_data(con, recordlist):
        cur = con.cursor()
        SQL = f'{eHSQL.SQLLabFields}' + 'and cs.case_id in ' + recordlist
        cur.execute(SQL)
        global df
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            df = pd.read_sql(SQL, con)
            display(df)
            
            df_test = df.pivot(index = ['INVRESULTUNID'], columns = 'NAME', values=['VALUE'])
            display(df_test)
            df_test.columns = df_test.columns.droplevel()
            df_test = df_test.rename_axis(None, axis = 1)
            df = df.drop(columns = ['NAME', 'VALUE']).drop_duplicates()
            
            df = pd.merge(df, df_test, how ='outer', on = 'INVRESULTUNID')
            
            display(df)

            df['record_no'] = np.arange(len(df))+1
            rcount = df.pop('record_no')
            df.insert(0, rcount.name, rcount)
            df['INVUNID'] = df['INVUNID'].astype('str')
            df['binary'] = df['INVUNID'].shift(1, fill_value = df['INVUNID'].head(1)) != df['INVUNID']
            df['parent_id'] = df['INVUNID'].groupby((df['binary']).cumsum()).cumcount()+1
            df['U_ID'] = df.groupby('INVUNID', sort=False).ngroup()+1
            df['rollup_no'] = np.nan
            df.loc[df['parent_id'] == 1, 'rollup_no'] = df['record_no']
            df['rollup_no'] = df['rollup_no'].ffill()
            df['rollup_no'] = df['rollup_no'].astype('int')
            #df['rollup_no'] = df['rollup_no'].ffill()
            
            #df_test = df_test.reset_index()
            
            df.set_index('record_no', inplace = True)

            #df['parent_id'] = df.groupby([''])
            #rcount = df.pop('parent_id')
            #df.insert(1, rcount.name, rcount)
            
            #dfshift = pd.DataFrame(df['INVUNID'])
            #dfshift = dfshift.shift(1).append(dfshift.iloc[[-1]]).reset_index(drop = True)

            df = df.drop(columns = ['binary'])
            
            col = df.pop('U_ID')
            df.insert(1, col.name, col)
            
            col = df.pop('parent_id')
            df.insert(2, col.name, col)
            
            col = df.pop('rollup_no')
            df.insert(3, col.name, col)
            
            recordadded = False
            isGenoType = False
            genotypeid = ['61199-6','34700-5','45176-5','45175-7','33630-5','53798-5','90903-6','81122-4']
            
            #dfshift = pd.DataFrame(df['INVUNID'])
            #dfshift = dfshift.shift(1).append(dfshift.iloc[[-1]]).reset_index(drop = True)
            
            df['doc_type'] = "004"
            df['source'] = "A05.03"
            df['surv_method'] = "P"
            
            
            datelist = ('CREATE_DATE', 'MODIFICATION_DATE', 'BIRTH_DATE')
            date_format = "%m/%d/%Y"
            
            for date in enumerate(datelist):
                if date[1] in df:
                    df[date[1]] = df[date[1]].dt.strftime(date_format)
                else:
                    continue
                    
            df['enter_dt'] = df['CREATE_DATE']
            df['rcvd_dt'] = df['CREATE_DATE']
            df['m_name'] = ''
            df.loc[df['parent_id'] == 1, 'f_name'] = df['FIRST_NAME']
            df.loc[df['parent_id'] == 1, 'l_name'] = df['LAST_NAME']
            df.loc[df['parent_id'] == 1, 'birth_dt'] = df['BIRTH_DATE']
            df.loc[df['parent_id'] == 1, 'name_use_cd'] = 'L'
            df['isGenoType'] = False
            
            df['facility_name'] = ''
            df['facility_phone'] = ''
            df['facility_city'] = ''
            df['facility_st'] = ''
            df['facility_ctry'] = ''
            df['facility_usd'] = ''
            df['facility_type'] = ''
            
            df['specimen_id'] = ''
            df['lab_specific_note'] = ''
            df['comments'] = ''
            df['notes'] = ''
            
            df['complete_dt'] = ''
            df['name_use_cd'] = ''
            df['addr_type_cd'] = ''
            df['address_dt'] = ''
            df['state'] = ''
            df['zip_code'] = ''
            df['addr_1'] = ''
            df['addr_2'] = ''
            df['city'] = ''
            df['cnty'] = ''
            df['cntry'] = ''
            df['cntry_usd'] = ''
            df['phone'] = ''
            df['birth_sex'] = ''
            df['age_at_hiv'] = ''
            df['age_at_aids'] = ''
            df['birth_dt'] = ''
            df['birth_cntry'] = ''
            df['birth_usd'] = ''
            df['vital_status'] = ''
            df['death_dt'] = ''
            df['death_st'] = ''
            df['gender'] = ''
            df['ethnicity'] = ''
            df['race1'] = ''
            df['race2'] = ''
            df['race3'] = ''
            df['race4'] = ''
            df['race5'] = ''
            df['event_cd'] = ''
            df['swm'] = ''
            df['swf'] = ''
            df['idu'] = ''
            df['rcvd_clot_factor'] = ''
            df['factor_type'] = ''
            df['factor_dt'] = ''
            df['sexidu'] = ''
            df['sexbi'] = ''
            df['sexhemo'] = ''
            df['sex_transf'] = ''
            df['sex_transpl'] = ''
            df['sex_aidshiv'] = ''
            df['rcvd_transf'] = ''
            df['first_trans_dt'] = ''
            df['last_trans_dt'] = ''
            df['rcvd_transpl'] = ''
            df['hlth_worker'] = ''
            df['occupation'] = ''
            df['other_risk'] = ''
            df['nir'] = ''
            df['nir_dt'] = ''
            df['acute_suspect'] = ''
            df['acute_symp'] = ''
            df['acute_symp_dt'] = ''
            df['acute_other'] = ''
            df['acute_other_specify'] = ''
            df['acute_other_dt'] = ''
            df['refer_obgyn'] = ''
            df['pregnant'] = ''
            df['live_infant'] = ''
            df['interview_dt'] = ''
            df['first_pos_test_dt'] = ''
            df['last_negtest_dt'] = ''
            
            try:
                df['notes'] = df['Notes']
            except KeyError:
                df['notes'] = ''
                pass
            try:
                df['lab_clia_no'] = df['CLIA']
            except KeyError:
                df['lab_clia_no'] = ''
                pass
            try:
                df['accession_no'] = df['SpecimenNumber']
            except KeyError:
                df['accession_no'] = ''
                pass
            try:
                df['colldate'] = df['SpecimenDate']
            except KeyError:
                df['colldate'] = ''
                pass
                
            df['result_dt'] = ''
            df['specimen_recieve_dt'] = ''
            df['specimentype'] = ''
            df['lab_result'] = ''
            df['lab_units'] = ''
            
            df['lab_test_name'] = df['Test']
            df['genotype_sequence'] = ''
            df['starhs_sample_id'] = ''
            df['sreason'] = ''
                
            #Interpretation condition values
            interpret1 = ['20447-9','25836-8','29541-0','24013-5','25835-0','59419-2','70241-5']
            statuslist = ['24013-5','29541-0','25835-0']
            interpret2 = ['29541-0','24013-5']

            
            PCT = ['8128-1','8123-2','32516-7']
            CNT = ['8127-3','24467-3','32515-9']
            CML = ['20447-9','25836-8','24013-5','59419-2','70241-5','25835-0']
            LOG = ['29541-0']
            
            units = ['PCT','CNT','Copies/ML','Log']
            
            POS = ['10828004','G-A200','11214006','G-A497','261420005','260373001']
            NEG = ['260415000','260385009']
            IND = ['82334004']
            
            result = ['POS', 'NEG', 'IND']
            
            #Identify HIV/STD cases
            for gid in enumerate(genotypeid):
                df.loc[df['Test'] == str(gid[1]), 'isGenoType'] = True
            
            #Identify UOM (Unit of Measurement)
            for vals in enumerate(PCT):
                df.loc[df['Test'] == str(vals[1]), 'lab_units'] = str(units[0])
            
            for vals in enumerate(CNT):
                df.loc[df['Test'] == str(vals[1]), 'lab_units'] = str(units[1])
              
            for vals in enumerate(CML):
                df.loc[df['Test'] == str(vals[1]), 'lab_units'] = str(units[2])
            
            for vals in enumerate(LOG):
                df.loc[df['Test'] == str(vals[1]), 'lab_units'] = str(units[3])
                
            df.loc[(df['Comments'] != 'None') | (df['notes'] != ''),'lab_specific_note'] = df['Comments'][0:100]
            df.loc[(df['Comments'] != 'None') | (df['notes'] != ''),'comments'] = df['Comments'][0:255]
            
            df.loc[(df['ResultValue'].str.contains('ND00')) & (df['isGenoType'] == True) & (df['CITY'] != 'None'),'genotype_sequence'] = df['EXTENDED_VALUE']
            df.loc[(df['ResultValue'].str.contains('ND00')) & (df['isGenoType'] == True) & (df['CITY'] != 'None'),'lab_units'] = 'BP'
            
            df.loc[(df['isGenoType'] == True) & (df['CITY'] != 'None'), 'genotype_sequence'] = df['EXTENDED_VALUE']
            df.loc[(df['isGenoType'] == True) & (df['CITY'] != 'None'),'lab_units'] = 'BP'
            
            #Identify UOM (Unit of Measurement)
            for vals in enumerate(POS):
                df.loc[df['Result'] == str(vals[1]), 'lab_result'] = str(result[0])
            for vals in enumerate(NEG):
                df.loc[df['Result'] == str(vals[1]), 'lab_result'] = str(result[1])
            for vals in enumerate(IND):
                df.loc[df['Result'] == str(vals[1]), 'lab_result'] = str(result[2])
                    
            if df['ResultValue'].iteritems() != 'None':
                if df['ResultValue'].str.contains(';').items():
                    df['ResultValue'] = df['ResultValue'].str.replace('^[^;]*;','').str.replace('[A-Z]','')
            
            if df['ResultValue'].iteritems() != 'None':
                if df['ResultValue'].str.contains('<').items():
                    df['ResultValueTemp'] = df['ResultValue'].str.replace("<","")
                    
            df.loc[(df['lab_result'] == 'None') & (df['isGenoType'] == False), 'lab_result'] = df['CASE_ID']
            df.loc[(df['lab_result'] == '') ,'lab_result'] = df['ResultValueTemp']

            for vals in enumerate(interpret1):
                df.loc[(df['Test'] == str(vals[1])) & (df['ResultValue'].str.contains('<')), 'lab_result'] = '0'
                df.loc[(df['Test'] == str(vals[1])) & (df['ResultValue'].str.contains('<')), 'interpretation'] = '<'
                
                df['result_range_upper'] = 10000000
                df.loc[(df['Test'] == str(vals[1])) & (df['ResultValue'].str.contains('<')) & ('.' not in df['lab_result']), 'result_range_lower'] = df['lab_result']
                
            for vals in enumerate(statuslist):
                df.loc[(df['Test'] == str(vals[1])) & (df['ReferenceRange'].str.contains('DETECTED')),'lab_result'] = '0'
                df.loc[(df['Test'] == str(vals[1])) & (df['ReferenceRange'].str.contains('NOT DETECTED')),'lab_result'] = None
                df.loc[(df['Test'] == str(vals[1])) & (df['ReferenceRange'].str.contains('NOT QUANTIFIED')),'lab_result'] = None
            

            
            for vals in enumerate(interpret2):
                df.loc[(df['Test'] == str(vals[1])) & (df['ResultValueTemp'].iteritems() != 'None'),'interpretation'] = '<'
                df.loc[(df['Test'] == str(vals[1])) & (df['ResultValueTemp'].iteritems() != 'None') & (df['ResultValueTemp'] >= '0'),'interpretation'] = '='
            
            df['result_range_lower'] = 20
            df.loc[df['Test'] == '59419-2', 'result_range_lower'] = 40
            df['result_range_upper'] = 10000000
            
            df.loc[(df['ResultValueTemp'] != 'None') & (df['ResultValueTemp'] < '20'), 'interpretation'] = '<'
            df.loc[((df['ResultValueTemp'] != 'None') & (df['ResultValueTemp'] >= '20') & (df['ResultValueTemp'] <= '10000000')), 'interpretation'] = '='
            df.loc[(df['ResultValueTemp'] != 'None') & (df['ResultValueTemp'] > '10000000'), 'interpretation'] = '>'
            
            df.loc[df['result_range_lower'] == '1.6', 'result_range_lower'] = '' 
            df.loc[df['lab_test_name'] == '20447-9', 'lab_test_name'] = '59419'            
            df.loc[((df['lab_test_name'] == '20447-9') | (df['lab_test_name'] == '25836-8') | (df['lab_test_name'] == '29541-0') | (df['lab_test_name'] == '70241-5') | (df['lab_test_name'] == '25835-0')) , 'lab_test_name'] = '74854'        
            df.loc[((df['lab_test_name'] == '24467-3') | (df['lab_test_name'] == '32515-9')) , 'lab_test_name'] = '24467'  
            df.loc[((df['lab_test_name'] == '8123-2') | (df['lab_test_name'] == '32516-7')) , 'lab_test_name'] = '32516-7'
            df['lab_result'] = df['lab_result'].str.replace('=','')
                    
            display(df)
                    
            df = df.drop(columns = ['ResultValueTemp',
                                    'INVUNID',
                                    'CASE_ID',
                                    'INVRESULTUNID',
                                    'STREET1',
                                    'STREET2',
                                    'CITY',
                                    'STATE',
                                    'POSTAL_CODE',
                                    'CREATE_DATE', 
                                    'MODIFICATION_DATE', 
                                    'ReferenceRange', 
                                    'Result', 
                                    'ResultStatus', 
                                    'ResultUnits', 
                                    'TestLocalCode', 
                                    'TestLocalDescription', 
                                    'TestValue',
                                    'FIRST_NAME', 
                                    'LAST_NAME', 
                                    'BIRTH_DATE'])
            
            df = df[['U_ID',
                     'parent_id',
                     'rollup_no',
                     'doc_type',
                     'STATENO',
                     'rcvd_dt',
                     'source',
                     'surv_method',
                     'name_use_cd',
                     'f_name',
                     'm_name',
                     'l_name',
                     'birth_dt',
                     'f_name',
                     'l_name',
                     'birth_dt',
                     'facility_name',
                     'facility_type',
                     'facility_city',
                     'facility_st',
                     'facility_ctry',
                     'facility_usd',
                     'facility_phone',
                     'lab_clia_no',
                     'specimen_id',
                     'accession_no',
                     'colldate',
                     'result_dt',
                     'specimen_recieve_dt',
                     'lab_test_name',
                     'specimentype',
                     'lab_result',
                     'lab_units',
                     'interpretation',
                     'result_range_lower',
                     'result_range_upper',
                     'genotype_sequence',
                     'starhs_sample_id',
                     'sreason',
                     'lab_specific_note',
                     'comments']]
            
            with pd.option_context('display.max_rows', None, 'display.max_columns', None):
                display(df)
            #display(dfshift)
            
        #print(f'{eHSQL.SQLLabFields}' + 'and cs.case_id in ' + recordlist)
        output = cur.fetchall()
        #print(output)
        #print(f'\n{output}\n', f'\n{type(output)}\n' ,end = "\r")
        logging.info(f'\nData Succesfully Prepped.\n')
        cur.close()
        con.commit()
        return df
    
    def Update_Last_Extract():
        cur = con.cursor()
        SQL = f'{eHSQL.SQLDateUpdate}' + recordlist
        cur.execute(SQL)
        global df2
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            df2 = pd.read_sql(SQL, con)
            
        display(df2)
        
        #for ids in enumerate(df2['UNID']):
        #    if ((ids[1] != 'None') | (ids[1] != 0)):
        #        dtu = "Update NDDH_EDSS.IDS_Answer set Value = '" + date.today().strftime('%dd/%mm/%Y') + "', Date_Value = CURRENT_TIMESTAMP where UNID = '" + ids[1] + "'"
        #        cur.execute(dtu)
        #        cur.close()
        #        con.commit()
        
        cur.close()
        con.commit()
        return df2
    
    def GetQuestionSetUNID():
        cur = con.cursor()
        SQL = f'{eHSQL.SQLQuestionID}' + recordlist
        cur.execute(SQL)
        global df3
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            df3 = pd.read_sql(SQL, con)
            
        display(df3)
        
        cur.close()
        con.commit()
        return df3
    
    def GetGlobalUNID():
        cur = con.cursor()
        SQL = f'{eHSQL.SQLGlobalID}'
        cur.execute(SQL)
        global df4
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            df4 = pd.read_sql(SQL, con)
            
        display(df4)

        cur.close()
        con.commit()
        return df4

    open_connection()
    conn = True
    connection_test()
    close_connection()
    conn = False
    
    open_connection()
    conn = True
    records = False
    if tablecount(con) == False:
        print("No New Records.\n", end = "\r")
    else:
        print("\nRecords ready for processing.\n", end = "\r")
        tablecontent(con)
        get_data(con, recordlist)
        Update_Last_Extract()
        GetQuestionSetUNID()
        GetGlobalUNID()
        df.to_csv(f'eHARSLabOut_{str(date.today())}.csv', sep = '\t', encoding = 'utf-8', header = True)
        #for cases in caselist
        #get_data()en

except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    print(e, 'line: ', exc_tb.tb_lineno,'\n\n', traceback.format_exc())
    pass

except (Exception, O.Error) as dberr:
    print(dberr)
    logging.exception("message")
    pass

#except Exception as err:
     #for receivers in eh.settings.ReceiverList:
            #SendEmailMessage(receivers, str(e))

finally:
    logging.info(f'\nLOG END: {datetime.now()}\n')
    if conn == True:
        close_connection()

Connection established to:  Oracle Database 12c EE Extreme Perf Release 12.2.0.1.0 - 64bit Production

6 Existing Records

Records ready for processing.


,CASE_ID,UNID
0,100000571,181062204
1,100000008,2921245
2,100000296,90866775
3,100000381,122928347
4,100000518,166582986
5,100006970,574151293


,INVUNID,CASE_ID,STATENO,INVRESULTUNID,CREATE_DATE,MODIFICATION_DATE,FIRST_NAME,LAST_NAME,BIRTH_DATE,NAME,VALUE,EXTENDED_VALUE,STREET1,STREET2,CITY,STATE,POSTAL_CODE
0,2921321,100000008,201801,2921313,2018-02-01 09:00:20.010,2018-02-01 09:00:33.054,Layla,Bunny,1985-01-05,Result,G-A385,None,123 Seasme St.,None,Bismarck,ND,58503
1,2921321,100000008,201801,2921313,2018-02-01 09:00:20.010,2018-02-01 09:00:33.054,Layla,Bunny,1985-01-05,Test,14092-1,None,123 Seasme St.,None,Bismarck,ND,58503
2,1067036744,100000296,201901,1067036732,2022-09-15 09:11:00.408,2022-09-15 09:11:00.408,Fred,Flintstone,2000-01-11,ReferenceRange,30-56,None,1234 Sweet Ave,None,Fargo,ND,58103
3,1067036744,100000296,201901,1067036732,2022-09-15 09:11:00.408,2022-09-15 09:11:00.408,Fred,Flintstone,2000-01-11,Result,ND000009,None,1234 Sweet Ave,None,Fargo,ND,58103
4,1067036744,100000296,201901,1067036732,2022-09-15 09:11:00.408,2022-09-15 09:11:00.408,Fred,Flintstone,2000-01-11,ResultStatus,Final Results,None,1234 Sweet Ave,None,Fargo,ND,58103
5,1067036744,100000296,201901,1067036732,2022-09-15 09:11:00.408,2022-09-15 09:11:00.408,Fred,Flintstone,2000-01-11,ResultUnits,%,None,1234 Sweet Ave,None,Fargo,ND,58103
6,1067036744,100000296,201901,1067036732,2022-09-15 09:11:00.408,2022-09-15 09:11:00.408,Fred,Flintstone,2000-01-11,ResultValue,ND000009; 35,None,1234 Sweet Ave,None,Fargo,ND,58103
7,1067036744,100000296,201901,1067036732,2022-09-15 09:11:00.408,2022-09-15 09:11:00.408,Fred,Flintstone,2000-01-11,Test,8123-2,None,1234 Sweet Ave,None,Fargo,ND,58103
8,1067036744,100000296,201901,1067036732,2022-09-15 09:11:00.408,2022-09-15 09:11:00.408,Fred,Flintstone,2000-01-11,TestLocalCode,3319,None,1234 Sweet Ave,None,Fargo,ND,58103
9,1067036744,100000296,201901,1067036732,2022-09-15 09:11:00.408,2022-09-15 09:11:00.408,Fred,Flintstone,2000-01-11,TestLocalDescription,% CD4 (T Cells),None,1234 Sweet Ave,None,Fargo,ND,58103


VALUE  \
NAME                                                    Comments   
INVRESULTUNID                                                      
2921313                                                      NaN   
992782680                                                    NaN   
1066930003     HIV RNA level detected is <20 copies/ml (<1.30...   
1066930018     SPECIMEN SOURCE: BLOOD SPECIMEN SPECIMEN SITE:...   
1066979840     Interpretation of results: SUSC= Susceptible P...   
1066980140     SPECIMEN SOURCE: BLOOD SPECIMEN SPECIMEN SITE:...   
1066980169                                                   NaN   
1067036709                                                   NaN   
1067036732                                                   NaN   
1067036778                                                   NaN   
1067036788                                                   NaN   

                                                                       \
NAME                          ReferenceRange    Result   ResultStatus   
INVRESULTUNID                                                           
2921313                                  NaN    G-A385            NaN   
992782680                                NaN       NaN            NaN   
1066930003     Reference Value: Not Detected  ND000009  Final Results   
1066930018     Reference Value: Not Detected  ND000009  Final Results   
1066979840                               NaN  ND000024  Final Results   
1066980140                             30-66  ND000009  Final Results   
1066980169                         400-2,156  ND000009  Final Results   
1067036709                          497-2267  ND000009  Final Results   
1067036732                             30-56  ND000009  Final Results   
1067036778                             32-64  ND000009  Final Results   
1067036788                          365-1437  ND000009  Final Results   

                                                                    \
NAME          ResultUnits       ResultValue     Test TestLocalCode   
INVRESULTUNID                                                        
2921313               NaN               NaN  14092-1           NaN   
992782680             NaN               255   8127-3           NaN   
1066930003            NaN     ND000009; <20  20447-9           NaN   
1066930018            NaN   ND000009; <1.30  29541-0           NaN   
1066979840            NaN  ND000024; INTERP  34700-5         37216   
1066980140            NaN      ND000009; 19  32516-7           NaN   
1066980169            NaN     ND000009; 290  32515-9           NaN   
1067036709      cells/mcL     ND000009; 730  24467-3          3325   
1067036732              %      ND000009; 35   8123-2          3319   
1067036778              %                13   8123-2          3319   
1067036788      cells/mcL     ND000009; 133  24467-3          3325   

                                                         \
NAME                               TestLocalDescription   
INVRESULTUNID                                             
2921313                                             NaN   
992782680                                           NaN   
1066930003                                          NaN   
1066930018                                          NaN   
1066979840     HIV-1 Genotypic PR-RT Drug Resistance, P   
1066980140                                          NaN   
1066980169                                          NaN   
1067036709                                CD4 (T Cells)   
1067036732                              % CD4 (T Cells)   
1067036778                              % CD4 (T Cells)   
1067036788                                CD4 (T Cells)   

                                                                  
NAME                                                   TestValue  
INVRESULTUNID                                                     
2921313                                                      NaN  
9927826

,INVUNID,CASE_ID,STATENO,INVRESULTUNID,CREATE_DATE,MODIFICATION_DATE,FIRST_NAME,LAST_NAME,BIRTH_DATE,EXTENDED_VALUE,STREET1,STREET2,CITY,STATE,POSTAL_CODE,Comments,ReferenceRange,Result,ResultStatus,ResultUnits,ResultValue,Test,TestLocalCode,TestLocalDescription,TestValue
0,2921321,100000008,201801,2921313,2018-02-01 09:00:20.010,2018-02-01 09:00:33.054,Layla,Bunny,1985-01-05,None,123 Seasme St.,None,Bismarck,ND,58503,NaN,NaN,G-A385,NaN,NaN,NaN,14092-1,NaN,NaN,NaN
1,1067036744,100000296,201901,1067036732,2022-09-15 09:11:00.408,2022-09-15 09:11:00.408,Fred,Flintstone,2000-01-11,None,1234 Sweet Ave,None,Fargo,ND,58103,NaN,30-56,ND000009,Final Results,%,ND000009; 35,8123-2,3319,% CD4 (T Cells),8123-2; CD3+CD4+ Cells NFr Bld
2,1067036744,100000296,201901,1067036709,2022-09-15 09:11:01.326,2022-09-15 09:11:01.326,Fred,Flintstone,2000-01-11,None,1234 Sweet Ave,None,Fargo,ND,58103,NaN,497-2267,ND000009,Final Results,cells/mcL,ND000009; 730,24467-3,3325,CD4 (T Cells),24467-3; CD3+CD4+ Cells # Bld
3,1067036789,100000296,201901,1067036778,2022-09-15 09:10:56.430,2022-09-15 09:15:00.032,Fred,Flintstone,2000-01-11,None,1234 Sweet Ave,None,Fargo,ND,58103,NaN,32-64,ND000009,Final Results,%,13,8123-2,3319,% CD4 (T Cells),8123-2; CD3+CD4+ Cells NFr Bld
4,1067036789,100000296,201901,1067036788,2022-09-15 09:10:58.253,2022-09-15 09:10:58.253,Fred,Flintstone,2000-01-11,None,1234 Sweet Ave,None,Fargo,ND,58103,NaN,365-1437,ND000009,Final Results,cells/mcL,ND000009; 133,24467-3,3325,CD4 (T Cells),24467-3; CD3+CD4+ Cells # Bld
5,1066979847,100000571,120594,1066979840,2022-09-15 08:55:53.659,2022-09-15 08:55:53.659,Contact_test,Contact_test2,2009-07-01,None,Po Box 555,None,Fargo,ND,58107,Interpretation of results: SUSC= Susceptible P...,NaN,ND000024,Final Results,NaN,ND000024; INTERP,34700-5,37216,"HIV-1 Genotypic PR-RT Drug Resistance, P",34700-5; HIV RT+PR Mut Tested Islt
6,1066980171,100000571,120594,1066980169,2022-09-15 08:56:20.594,2022-09-15 08:56:20.594,Contact_test,Contact_test2,2009-07-01,None,Po Box 555,None,Fargo,ND,58107,NaN,"400-2,156",ND000009,Final Results,NaN,ND000009; 290,32515-9,NaN,NaN,32515-9; CD3+CD4+ (T4 helper) cells [#/volume]...
7,1066980171,100000571,120594,1066980140,2022-09-15 08:56:19.932,2022-09-15 08:56:19.932,Contact_test,Contact_test2,2009-07-01,None,Po Box 555,None,Fargo,ND,58107,SPECIMEN SOURCE: BLOOD SPECIMEN SPECIMEN SITE:...,30-66,ND000009,Final Results,NaN,ND000009; 19,32516-7,NaN,NaN,32516-7; CD3+CD4+ (T4 helper) cells/100 cells ...
8,1066930024,100000571,120594,1066930003,2022-09-14 16:25:59.336,2022-09-14 16:25:59.336,Contact_test,Contact_test2,2009-07-01,None,Po Box 555,None,Fargo,ND,58107,HIV RNA level detected is <20 copies/ml (<1.30...,Reference Value: Not Detected,ND000009,Final Results,NaN,ND000009; <20,20447-9,NaN,NaN,20447-9; HIV 1 RNA [#/volume] (viral load) in ...
9,1066930024,100000571,120594,1066930018,2022-09-14 16:25:57.316,2022-09-14 16:25:57.316,Contact_test,Contact_test2,2009-07-01,None,Po Box 555,None,Fargo,ND,58107,SPECIMEN SOURCE: BLOOD SPECIMEN SPECIMEN SITE:...,Reference Value: Not Detected,ND000009,Final Results,NaN,ND000009; <1.30,29541-0,NaN,NaN,29541-0; HIV 1 RNA [Log #/volume] (viral load)...


...

,INVUNID,U_ID,parent_id,rollup_no,CASE_ID,STATENO,INVRESULTUNID,CREATE_DATE,MODIFICATION_DATE,FIRST_NAME,LAST_NAME,BIRTH_DATE,EXTENDED_VALUE,STREET1,STREET2,CITY,STATE,POSTAL_CODE,Comments,ReferenceRange,Result,ResultStatus,ResultUnits,ResultValue,Test,TestLocalCode,TestLocalDescription,TestValue,doc_type,source,surv_method,enter_dt,rcvd_dt,m_name,f_name,l_name,birth_dt,name_use_cd,isGenoType,facility_name,facility_phone,facility_city,facility_st,facility_ctry,facility_usd,facility_type,specimen_id,lab_specific_note,comments,notes,complete_dt,addr_type_cd,address_dt,state,zip_code,addr_1,addr_2,city,cnty,cntry,cntry_usd,phone,birth_sex,age_at_hiv,age_at_aids,birth_cntry,birth_usd,vital_status,death_dt,death_st,gender,ethnicity,race1,race2,race3,race4,race5,event_cd,swm,swf,idu,rcvd_clot_factor,factor_type,factor_dt,sexidu,sexbi,sexhemo,sex_transf,sex_transpl,sex_aidshiv,rcvd_transf,first_trans_dt,last_trans_dt,rcvd_transpl,hlth_worker,occupation,other_risk,nir,nir_dt,acute_suspect,acute_symp,acute_symp_dt,acute_other,acute_other_specify,acute_other_dt,refer_obgyn,pregnant,live_infant,interview_dt,first_pos_test_dt,last_negtest_dt,lab_clia_no,accession_no,colldate,result_dt,specimen_recieve_dt,specimentype,lab_result,lab_units,lab_test_name,genotype_sequence,starhs_sample_id,sreason,ResultValueTemp,interpretation,result_range_upper,result_range_lower
record_no,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2921321,1,1,1,100000008,201801,2921313,02/01/2018,02/01/2018,Layla,Bunny,01/05/1985,None,123 Seasme St.,None,Bismarck,ND,58503,NaN,NaN,G-A385,NaN,NaN,NaN,14092-1,NaN,NaN,NaN,004,A05.03,P,02/01/2018,02/01/2018,,Layla,Bunny,,,False,,,,,,,,,NaN,NaN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,NaN,,14092-1,,,,NaN,NaN,10000000,20
2,1067036744,2,1,2,100000296,201901,1067036732,09/15/2022,09/15/2022,Fred,Flintstone,01/11/2000,None,1234 Sweet Ave,None,Fargo,ND,58103,NaN,30-56,ND000009,Final Results,%,35,8123-2,3319,% CD4 (T Cells),8123-2; CD3+CD4+ Cells NFr Bld,004,A05.03,P,09/15/2022,09/15/2022,,Fred,Flintstone,,,False,,,,,,,,,NaN,NaN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,35,PCT,32516-7,,,,35,<,10000000,20
3,1067036744,2,2,2,100000296,201901,1067036709,09/15/2022,09/15/2022,Fred,Flintstone,01/11/2000,None,1234 Sweet Ave,None,Fargo,ND,58103,NaN,497-2267,ND000009,Final Results,cells/mcL,730,24467-3,3325,CD4 (T Cells),24467-3; CD3+CD4+ Cells # Bld,004,A05.03,P,09/15/2022,09/15/2022,,NaN,NaN,,,False,,,,,,,,,NaN,NaN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,730,CNT,24467,,,,730,<,10000000,20
4,1067036789,3,1,4,100000296,201901,1067036778,09/15/2022,09/15/2022,Fred,Flintstone,01/11/2000,None,1234 Sweet Ave,None,Fargo,ND,58103,NaN,32-64,ND000009,Final Results,%,13,8123-2,3319,% CD4 (T Cells),8123-2; CD3+CD4+ Cells NFr Bld,004,A05.03,P,09/15/2022,09/15/2022,,Fred,Flintstone,,,False,,,,,,,,,NaN,NaN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,13,PCT,32516-7,,,,13,>,10000000,20
5,1067036789,3,2,4,100000296,201901,1067036788,09/15/2022,09/15/2022,Fred,Flintstone,01/11/2000,None,1234 Sweet Ave,None,Fargo,ND,58103,NaN,365-1437,ND000009,Final Results,cells/mcL,133,24467-3,3325,CD4 (T Cells),24467-3; CD3+CD4+ Cells # Bld,004,A05.03,P,09/15/2022,09/15/2022,,NaN,NaN,,,False,,,,,,,,,NaN,NaN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,133,CNT,24467,,,,133,<,10000000,20
6,1066979847,4,1,6,100000571,120594,1066979840,09/15/2022,09/15/2022,Contact_test,Contact_test2,07/01/2009,None,Po Box 555,None,Fargo,ND,58107,Interpretation of results: SUSC= Susceptible P...,NaN,ND000024,Final Results,NaN,,34700-5,37216,"HIV-1 Genotypic PR-RT Drug Resistance, P",34700-5; HIV RT+PR Mut Tested Islt,004,A05.03,P,09/15/2022,09/15/2022,,Contact_test,Contact_test2,,,True,,,,,,,,,Interpretation of results: SUSC= Susceptible P...,Interpretation of results: SUSC= Susceptib

,U_ID,parent_id,rollup_no,doc_type,STATENO,rcvd_dt,source,surv_method,name_use_cd,f_name,m_name,l_name,birth_dt,f_name,l_name,birth_dt,facility_name,facility_type,facility_city,facility_st,facility_ctry,facility_usd,facility_phone,lab_clia_no,specimen_id,accession_no,colldate,result_dt,specimen_recieve_dt,lab_test_name,specimentype,lab_result,lab_units,interpretation,result_range_lower,result_range_upper,genotype_sequence,starhs_sample_id,sreason,lab_specific_note,comments
record_no,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,004,201801,02/01/2018,A05.03,P,,Layla,,Bunny,,Layla,Bunny,,,,,,,,,,,,,,,14092-1,,NaN,,NaN,20,10000000,,,,NaN,NaN
2,2,1,2,004,201901,09/15/2022,A05.03,P,,Fred,,Flintstone,,Fred,Flintstone,,,,,,,,,,,,,,,32516-7,,35,PCT,<,20,10000000,,,,NaN,NaN
3,2,2,2,004,201901,09/15/2022,A05.03,P,,NaN,,NaN,,NaN,NaN,,,,,,,,,,,,,,,24467,,730,CNT,<,20,10000000,,,,NaN,NaN
4,3,1,4,004,201901,09/15/2022,A05.03,P,,Fred,,Flintstone,,Fred,Flintstone,,,,,,,,,,,,,,,32516-7,,13,PCT,>,20,10000000,,,,NaN,NaN
5,3,2,4,004,201901,09/15/2022,A05.03,P,,NaN,,NaN,,NaN,NaN,,,,,,,,,,,,,,,24467,,133,CNT,<,20,10000000,,,,NaN,NaN
6,4,1,6,004,120594,09/15/2022,A05.03,P,,Contact_test,,Contact_test2,,Contact_test,Contact_test2,,,,,,,,,,,,,,,34700-5,,,BP,<,20,10000000,None,,,Interpretation of results: SUSC= Susceptible P...,Interpretation of results: SUSC= Susceptible P...
7,5,1,7,004,120594,09/15/2022,A05.03,P,,Contact_test,,Contact_test2,,Contact_test,Contact_test2,,,,,,,,,,,,,,,24467,,290,CNT,<,20,10000000,,,,NaN,NaN
8,5,2,7,004,120594,09/15/2022,A05.03,P,,NaN,,NaN,,NaN,NaN,,,,,,,,,,,,,,,32516-7,,19,PCT,<,20,10000000,,,,SPECIMEN SOURCE: BLOOD SPECIMEN SPECIMEN SITE:...,SPECIMEN SOURCE: BLOOD SPECIMEN SPECIMEN SITE:...
9,6,1,9,004,120594,09/14/2022,A05.03,P,,Contact_test,,Contact_test2,,Contact_test,Contact_test2,,,,,,,,,,,,,,,59419,,0,Copies/ML,<,20,10000000,,,,HIV RNA level detected is <20 copies/ml (<1.30...,HIV RNA level detected is <20 copies/ml (<1.30...


,UNID
0,2921275
1,90866802
2,122928376
3,166583010
4,181062232
5,574151350


,UNID
0,2921256
1,90866786
2,122928357
3,166582992
4,181062213
5,574151319


,RAWTOHEX(SYS_GUID())
0,EDA38ABD4F5E5684E053621A020A54A1


In [36]:
pd.read_csv('eHARSLabOut_2022-10-06.csv', sep='\t')

,record_no,U_ID,parent_id,rollup_no,doc_type,STATENO,rcvd_dt,source,surv_method,name_use_cd,f_name,m_name,l_name,birth_dt,f_name.1,l_name.1,birth_dt.1,facility_name,facility_type,facility_city,facility_st,facility_ctry,facility_usd,facility_phone,lab_clia_no,specimen_id,accession_no,colldate,result_dt,specimen_recieve_dt,lab_test_name,specimentype,lab_result,lab_units,interpretation,result_range_lower,result_range_upper,genotype_sequence,starhs_sample_id,sreason,lab_specific_note,comments
0,1,1,1,1,4,201801,02/01/2018,A05.03,P,L,Layla,NaN,Bunny,01/05/1985,Layla,Bunny,01/05/1985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14092-1,NaN,NaN,NaN,NaN,20,10000000,NaN,NaN,NaN,NaN,NaN
1,2,2,1,2,4,201901,09/15/2022,A05.03,P,L,Fred,NaN,Flintstone,01/11/2000,Fred,Flintstone,01/11/2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32516-7,NaN,35,PCT,<,20,10000000,NaN,NaN,NaN,NaN,NaN
2,3,2,2,2,4,201901,09/15/2022,A05.03,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24467,NaN,730,CNT,<,20,10000000,NaN,NaN,NaN,NaN,NaN
3,4,3,1,4,4,201901,09/15/2022,A05.03,P,L,Fred,NaN,Flintstone,01/11/2000,Fred,Flintstone,01/11/2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32516-7,NaN,13,PCT,>,20,10000000,NaN,NaN,NaN,NaN,NaN
4,5,3,2,4,4,201901,09/15/2022,A05.03,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24467,NaN,133,CNT,<,20,10000000,NaN,NaN,NaN,NaN,NaN
5,6,4,1,6,4,120594,09/15/2022,A05.03,P,L,Contact_test,NaN,Contact_test2,07/01/2009,Contact_test,Contact_test2,07/01/2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34700-5,NaN,,BP,<,20,10000000,NaN,NaN,NaN,Interpretation of results: SUSC= Susceptible P...,Interpretation of results: SUSC= Susceptible P...
6,7,5,1,7,4,120594,09/15/2022,A05.03,P,L,Contact_test,NaN,Contact_test2,07/01/2009,Contact_test,Contact_test2,07/01/2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24467,NaN,290,CNT,<,20,10000000,NaN,NaN,NaN,NaN,NaN
7,8,5,2,7,4,120594,09/15/2022,A05.03,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32516-7,NaN,19,PCT,<,20,10000000,NaN,NaN,NaN,SPECIMEN SOURCE: BLOOD SPECIMEN SPECIMEN SITE:...,SPECIMEN SOURCE: BLOOD SPECIMEN SPECIMEN SITE:...
8,9,6,1,9,4,120594,09/14/2022,A05.03,P,L,Contact_test,NaN,Contact_test2,07/01/2009,Contact_test,Contact_test2,07/01/2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74854,NaN,0,Log,<,20,10000000,NaN,NaN,NaN,SPECIMEN SOURCE: BLOOD SPECIMEN SPECIMEN SITE:...,SPECIMEN SOURCE: BLOOD SPECIMEN SPECIMEN SITE:...
9,10,6,2,9,4,120594,09/14/2022,A05.03,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59419,NaN,0,Copies/ML,<,20,10000000,NaN,NaN,NaN,HIV RNA level detected is <20 copies/ml (<1.30...,HIV RNA level detected is <20 copies/ml (<1.30...


In [9]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
import sys
import pandas as pd
import os
import cx_Oracle as O
import datetime as dt
import pyodbc
import logging, logging.handlers
import settings

class errhandler:

    def CreateEmailMessage(toAddress, message):
        messageDetails = MIMEMultipart()
        
        messageDetails['From'] = settings.FROMEMAIL
        messageDetails['To'] = toAddress
        messageDetails['Subject'] = "Error handler triggered during eHARS processing"
        
        messageDetails.attach(MIMEText(message, 'plain'))
        
        return messageDetails
    
    def SendEmailMessage(toAddress, message):
            
            messageDetails = CreateEmailMessage(toAddress, message)
            
            try:
                server = smtplib.SMTP(settings.SMTPHOST, settings.SMTPPORT)
                server.starttls()
                server.ehlo()
                server.sendmail(settings.FROMEMAIL, toAddress, messageDetails.as_string())
            except Exception as ex:
                print(ex)
            finally:
                server.quit()


In [11]:
import errhandler as eh

try:
    a = int(input("Please enter a number: "))
except Exception as e:
        for receivers in eh.settings.ReceiverList:
            SendEmailMessage(receivers, str(e))

Please enter a number: h


In [ ]:
#Current Handling method


import smtplib
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from os.path import basename
import os
from datetime import datetime
import datetime as dt
# Set working directory to load files by name.
abspath = os.path.abspath(__file__)
dname = os.path.dirname(abspath)
os.chdir(dname)

filedate = dt.date.today()
filedated = dt.date.today()
filedatey = filedate - dt.timedelta(days=1)
filedatey2 = filedatey - dt.timedelta(days=1)

filedate = filedate.strftime("%m-%d-%Y")
filedated = filedated.strftime("%#m-%d-%Y")
filedatey = filedatey.strftime("%m-%d-%Y")
filedatey2 = filedatey2.strftime("%m-%d-%Y")

mydate = datetime.now()
thismonth = mydate.strftime("%B")

#Establish SMTP Connection
s = smtplib.SMTP('apprelay1.nd.gov', 25) 
  
#Start TLS based SMTP Session
s.starttls() 
 
#To Create Email Message in Proper Format
msg = MIMEMultipart()

#email
msg['From'] = disease@nd.gov
msg['To'] = mbenz@nd.gov, bschram@nd.gov, djdavidson@nd.gov, kkruger@nd.gov, mahowell@nd.gov, memoe@nd.gov, mdethloff@nd.gov, tjmiller@nd.gov, ddwilke@nd.gov, nwehbi@nd.gov, covidinfo@nd.gov, mnowatzki@nd.gov, tkmiller@nd.gov, gnjau@nd.gov, kpinks@nd.gov,  laskew@nd.gov, afolson@nd.gov
msg['Subject'] = filedate + " WebSite Numbers - Automated Email"
#body
message = """
Todays Website Numbers.  Open the bubbles_xx-xx-xxxx.csv to view the aggregate numbers.
"""
#Add Message To Email Body
msg.attach(MIMEText(message, 'html'))
#attach files
fileName = 'Labvolume_Output\\' + filedate + '\\LabVolume_'+ filedatey + '.xlsx'
file = open(fileName, "rb")
fileBaseName = basename(fileName)
part = MIMEApplication(file.read(), Name = fileBaseName)
part.add_header('Content-Disposition', 'attachment; filename="' + fileBaseName + '"')
msg.attach(part)

#To Send the Email
s.send_message(msg)
  
#Terminating the SMTP Session
s.quit()


In [3]:
import sys
import warnings
import pandas as pd
import numpy as np
import os
import cx_Oracle as O
import datetime as dt
import logging, logging.handlers, smtplib
import traceback
import smtplib
import sqlalchemy as sa
import pyodbc
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from os.path import basename
import os
from datetime import datetime, date
import settings
import errhandler as eh
import eHSQL
import re
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

con = None
try:
    global conn, recordlist
    conn = False
    logging.basicConfig(filename = 'eHARSExtract.log',
                       filemode='a',
                       format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                       datefmt='%H:%M:%S',
                       level=logging.DEBUG)
    logging.info(f'\nLOG START: {datetime.now()}\n')

    #Opens connection to the Oracle DB via the stored config file in the config folder
    def open_connection():
        global driver, database, uid, pwd, connection, con
        #connection = 'DRIVER={Oracle in instantclient_21_6};DBQ=NDEDSS_DEVsvc.itd.nd.gov;UID=NDDH_EDSS_Intr;PWD=QRSTUVW1'
        connection = f'{settings.username}/{settings.password}@{settings.servicename}'
        con = O.connect(connection)
        #con = pyodbc.connect(connection)
        return con

    def connection_test():
        cur = con.cursor()
        cur.execute("SELECT * FROM v$version")
        row = cur.fetchone()
        print("Connection established to: ", row[0])
        cur.close()
        con.commit()
        logging.info(f"\nConnection established to: {row[0]}\n")
        return

    def close_connection():
        con.close()
        return

    def tablecount(con):
        cur = con.cursor()
        cur.execute(eHSQL.SQLCaseCount)
        output = cur.fetchone()
        #print(output)
        if output[0] == 0:
            print(f'\n{output[0]} Existing Records\n', end = "\r")
            logging.info(f'\n{output[0]} Existing records\n')
            cur.close()
            con.commit()
            return False
        else:
            print(f'\n{output[0]} Existing Records\n', end = "\r")
            logging.info(f'\n{output[0]} Existing records\n')
            cur.close()
            con.commit()
            return True

    def tablecontent(con):
        global recordlist
        cur = con.cursor()
        SQL = eHSQL.SQLHIVData
        cur.execute(SQL)
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            df = pd.read_sql(SQL, con)
            display(df)
        output = cur.fetchall()
        output = output[0:]
        output = str(tuple(list(list(zip(*output))[0])))
        #print(output)
        #print(f'\nRecord List: {output}\n', end = "\r")
        logging.info(f'\nRecord List: {output}\n')
        recordlist = output
        cur.close()
        con.commit()
        return

    def get_data(con, recordlist):
        cur = con.cursor()
        SQL = f'{eHSQL.SQLLabFields}' + 'and cs.case_id in ' + recordlist
        cur.execute(SQL)
        global df
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            df = pd.read_sql(SQL, con)
            
            df_test = df.pivot(index = ['INVRESULTUNID'], columns = 'NAME', values=['VALUE'])
            df_test.columns = df_test.columns.droplevel()
            df_test = df_test.rename_axis(None, axis = 1)
            df = df.drop(columns = ['NAME', 'VALUE']).drop_duplicates()
            
            df = pd.merge(df, df_test, how ='outer', on = 'INVRESULTUNID')
            
            display(df)

            df['record_no'] = np.arange(len(df))+1
            rcount = df.pop('record_no')
            df.insert(0, rcount.name, rcount)
            df['INVUNID'] = df['INVUNID'].astype('str')
            df['binary'] = df['INVUNID'].shift(1, fill_value = df['INVUNID'].head(1)) != df['INVUNID']
            df['parent_id'] = df['INVUNID'].groupby((df['binary']).cumsum()).cumcount()+1
            df['U_ID'] = df.groupby('INVUNID', sort=False).ngroup()+1
            df['rollup_no'] = np.nan
            df.loc[df['parent_id'] == 1, 'rollup_no'] = df['record_no']
            df['rollup_no'] = df['rollup_no'].ffill()
            df['rollup_no'] = df['rollup_no'].astype('int')
            #df['rollup_no'] = df['rollup_no'].ffill()
            
            #df_test = df_test.reset_index()
            
            df.set_index('record_no', inplace = True)

            #df['parent_id'] = df.groupby([''])
            #rcount = df.pop('parent_id')
            #df.insert(1, rcount.name, rcount)
            
            #dfshift = pd.DataFrame(df['INVUNID'])
            #dfshift = dfshift.shift(1).append(dfshift.iloc[[-1]]).reset_index(drop = True)

            df = df.drop(columns = ['binary'])
            
            col = df.pop('U_ID')
            df.insert(1, col.name, col)
            
            col = df.pop('parent_id')
            df.insert(2, col.name, col)
            
            col = df.pop('rollup_no')
            df.insert(3, col.name, col)
            

            display(df)
            #display(dfshift)
            
        #print(f'{eHSQL.SQLLabFields}' + 'and cs.case_id in ' + recordlist)
        output = cur.fetchall()
        #print(output)
        #print(f'\n{output}\n', f'\n{type(output)}\n' ,end = "\r")
        logging.info(f'\nData Succesfully Prepped.\n')
        cur.close()
        con.commit()
        return df
    
    def get_QandA(con, recordlist):
        cur = con.cursor()
        SQL = f'{eHSQL.SQLQandA}' + recordlist
        cur.execute(SQL)
        global dfQA
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            dfQA = pd.read_sql(SQL, con)
            
            dfQA = dfQA.loc[dfQA['QUESTIONSET_ID'].str.contains('ADMINISTR')]
            
            df_test = dfQA.pivot(index = ['UNID','EXTERNAL_ID'], columns = ['QUESTIONSET_ID'], values=['VALUE','QUESTION_ID'])
            df_test.columns = df_test.columns.droplevel(1)
            df_test = df_test.rename_axis(None, axis = 1).reset_index()
            df_test = df_test.drop(columns = ['UNID'])
            
            #display(df_test)
            
            df_test = df_test.pivot(index = ['EXTERNAL_ID'], columns = ['QUESTION_ID'], values=['VALUE'])
            df_test.columns = df_test.columns.droplevel()
            df_test = df_test.rename_axis(None, axis = 1)
            
            #display(df_test)
             
            df_test = df_test.rename_axis(None, axis = 1).reset_index()
            dfQA = dfQA.drop(columns = ['UNID','QUESTIONSET_ID','QUESTION_ID','VALUE']).drop_duplicates()

            display(df_test)
            dfQA = pd.merge(dfQA, df_test, how ='outer', on = 'EXTERNAL_ID')
            
            #display(dfQA)

            dfQA['record_no'] = np.arange(len(dfQA))+1
            rcount = dfQA.pop('record_no')
            dfQA.insert(0, rcount.name, rcount)

            
            #display(dfQA)
            
            #df_test = df_test.reset_index()
            
            dfQA.set_index('record_no', inplace = True)

            df['parent_id'] = df.groupby([''])
            rcount = df.pop('parent_id')
            df.insert(1, rcount.name, rcount)

            dfQA = dfQA.drop(columns = ['binary'])
            
            col = dfQA.pop('U_ID')
            dfQA.insert(1, col.name, col)
            
            col = dfQA.pop('parent_id')
            dfQA.insert(2, col.name, col)
            
            col = dfQA.pop('rollup_no')
            dfQA.insert(3, col.name, col)
            
            display(dfQA)
    
    def Update_Last_Extract():
        cur = con.cursor()
        SQL = f'{eHSQL.SQLDateUpdate}' + recordlist
        cur.execute(SQL)
        global df2
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            df2 = pd.read_sql(SQL, con)
            
        display(df2)
        
        #for ids in enumerate(df2['UNID']):
        #    if ((ids[1] != 'None') | (ids[1] != 0)):
        #        dtu = "Update NDDH_EDSS.IDS_Answer set Value = '" + date.today().strftime('%dd/%mm/%Y') + "', Date_Value = CURRENT_TIMESTAMP where UNID = '" + ids[1] + "'"
        #        cur.execute(dtu)
        #        cur.close()
        #        con.commit()
        
        cur.close()
        con.commit()
        return df2
    
    def GetQuestionSetUNID():
        cur = con.cursor()
        SQL = f'{eHSQL.SQLQuestionID}' + recordlist
        cur.execute(SQL)
        global df3
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            df3 = pd.read_sql(SQL, con)
            
        display(df3)
        
        cur.close()
        con.commit()
        return df3
    
    def GetGlobalUNID():
        cur = con.cursor()
        SQL = f'{eHSQL.SQLGlobalID}'
        cur.execute(SQL)
        global df4
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            df4 = pd.read_sql(SQL, con)
            
        display(df4)

        cur.close()
        con.commit()
        return df4

    open_connection()
    conn = True
    connection_test()
    close_connection()
    conn = False
    
    open_connection()
    conn = True
    records = False
    if tablecount(con) == False:
        print("No New Records.\n", end = "\r")
    else:
        print("\nRecords ready for processing.\n", end = "\r")
        tablecontent(con)
        get_QandA(con, recordlist)
        Update_Last_Extract()
        GetQuestionSetUNID()
        GetGlobalUNID()
        #df.to_csv(f'eHARSLabQAOut_{str(date.today())}.csv', sep = '\t', encoding = 'utf-8', header = True)
        #for cases in caselist
        #get_data()en

except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    print(e, 'line: ', exc_tb.tb_lineno,'\n\n', traceback.format_exc())
    pass

except (Exception, O.Error) as dberr:
    print(dberr)
    logging.exception("message")
    pass

#except Exception as err:
     #for receivers in eh.settings.ReceiverList:
            #SendEmailMessage(receivers, str(e))

finally:
    logging.info(f'\nLOG END: {datetime.now()}\n')
    if conn == True:
        close_connection()

Connection established to:  Oracle Database 12c EE Extreme Perf Release 12.2.0.1.0 - 64bit Production

6 Existing Records

Records ready for processing.


,CASE_ID,UNID
0,100000571,181062204
1,100000008,2921245
2,100000296,90866775
3,100000381,122928347
4,100000518,166582986
5,100006970,574151293


,EXTERNAL_ID,ALL_HIV,ALL_STD,ASSIGNED_TO,CITY_HIDDEN,CLINICAL_EVENT_TYPE_LAB_TEST_DATE,CLINICAL_EVENT_TYPE_REPORT_DATE,DATE_FIRST_SUBMITTED,DATE_NOTIFICATION_SENT,DATE_TRANSMITTED_EHARS,DISEASE_HIV,DISEASE_STATUS_CONFIRMED,DISEASE_STATUS_UNKNOWN,DISEASE_TYPE_OTHER,EPI_ASSIGNMENT_OVERRIDE,EVENT_DATE,EVENT_DATE_OVERRIDE,EVENT_TYPE_OVERRIDE_LAB_TEST_DATE,FINAL_EPI_ASSIGNMENT,HEURISTIC_COUNTY_YES,INVESTIGATION_STATUS_1,MMWR_WEEK,MMWR_YEAR,OWNING_JURISDICTION,STATE_HIDDEN,STREET_HIDDEN,ZIP_HIDDEN,ZIP_LAB
0,100000008,YES,YES,srenton,Bismarck,LAB_TEST_DATE,NaN,01/01/1900,01/01/1900,01/01/1900,HIV,NaN,UNKNOWN,500,NaN,01/17/2018,NaN,NaN,srenton,YES,NEW,03,2018,Burleigh County,ND,123 Seasme St.,58503,NaN
1,100000296,YES,YES,kleeweyrauch,Fargo,LAB_TEST_DATE,NaN,01/01/1900,01/01/1900,01/01/1900,HIV,NaN,UNKNOWN,500,kleeweyrauch,03/04/2019,03/04/2019,LAB_TEST_DATE,kleeweyrauch,YES,NEW,10,2019,Cass County,ND,1234 Sweet Ave,58103,NaN
2,100000381,YES,YES,srenton,Bismarck,NaN,REPORT_DATE,01/01/1900,01/01/1900,01/01/1900,HIV,NaN,UNKNOWN,500,NaN,02/01/2019,NaN,NaN,srenton,NaN,NEW,05,2019,Burleigh County,ND,123 Sweet Ave,58501,NaN
3,100000518,YES,YES,srenton,NaN,NaN,REPORT_DATE,01/01/1900,01/01/1900,01/01/1900,HIV,NaN,UNKNOWN,500,NaN,06/06/2019,NaN,NaN,srenton,NaN,NEW,23,2019,NaN,ND,NaN,NaN,NaN
4,100000571,YES,YES,srenton,Fargo,LAB_TEST_DATE,NaN,01/01/1900,01/01/1900,01/01/1900,HIV,CONFIRMED,NaN,500,NaN,01/31/2022,NaN,NaN,srenton,YES,NEW,05,2022,Cass County,ND,Po Box 555,58107,58102
5,100006970,YES,YES,srenton,Bismarck,LAB_TEST_DATE,NaN,01/01/1900,01/01/1900,01/01/1900,HIV,NaN,UNKNOWN,500,NaN,08/23/2022,NaN,NaN,srenton,NaN,NEW,34,2022,Burleigh County,ND,Test Street,58501,NaN


'INVUNID' line:  300 

 Traceback (most recent call last):
  File "C:\Users\gmoye\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3621, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 136, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 163, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'INVUNID'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\gmoye\AppData\Local\Temp\1\ipykernel_11960\3336822144.py", line 300, in <cell line: 29>
    get_QandA(con, recordlist)
  File "C:\Users\gmoye\AppData\Local\Temp\1\ipykernel_11960\3336822144.py", line 201, in get_QandA
    dfQA['INVUNID'] = dfQA['INVUNID